In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

data = pd.read_csv('/Users/ChesterHuynh/learningml/Kaggle/Tutorials/house-prices-advanced-regression-techniques/train.csv')
data.dropna(axis=0, subset=['SalePrice'], inplace=True) # get rid of entries with no corresponding target
y = data.SalePrice
X = data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])
X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y.as_matrix(), test_size=0.25)

my_imputer = Imputer()
X_train = my_imputer.fit_transform(X_train)
X_test = my_imputer.fit_transform(X_test)

In [7]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(X_train, y_train, verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [9]:
# make predictions
predictions = my_model.predict(X_test)

from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, y_test)))

Mean Absolute Error : 19287.2768193


## Model Tuning

In [12]:
# XGBoost has a few parameters that can dramatically affect model's accuracy and training speed
# n_estimators = specifies how many times to go through the modeling cycle
# low n_est -> underfit, high n_est -> overfit
# typical values range from 100-1000, but depends on learning rate
# early_stopping_rounds = causes model to stop iterating when validating score stops improving
# since validation score not improving once can occur due to random chance,
# one needs to specify number of rounds of straight deterioration before stopping
# early_stopping_rounds = 5 seems reasonable

my_model = XGBRegressor(n_estimators=1000)
my_model.fit(X_train, y_train, early_stopping_rounds=5,
             eval_set = [(X_test, y_test)], verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [13]:
# learning_rate allows you to multiply predictions by a small number
# rather than simply adding up predictions from each component model, which
# reduces model's propensity to overfit, so that you can use higher n_est
# small learning_rate (and large # of estimators) will yield more accurate models,
# but may take longer to iterate through

my_model = XGBRegressor(n_estimators=1000, learning_rate = 0.05)
my_model.fit(X_train, y_train, early_stopping_rounds=5,
             eval_set = [(X_test, y_test)], verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [ ]:
# n_jobs = number of jobs running in parallel for large datasets